In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
f, axs = plt.subplots(1,3, figsize=[9,3])

for ax, idx in zip(axs, [1,2,3]):
    for geo in ['a32b32i0', 'a16b16i16', 'a0b0i32']:
        data = np.load(f'../analyses/diff/cp2k/{geo}-rho1.0753-msd-10-60ps/msd.npy')
        ax.plot(data[:,0], data[:,idx],'k')
        data = np.load(f'../analyses/diff/trial/{geo}-r1.08-msd-10-60ps/msd.npy')
        ax.plot(data[:,0], data[:,idx],'r')
        data = np.load(f'../analyses/diff/trial/{geo}-r1.08-msd-0-5ns/msd.npy')
        ax.plot(data[:,0], data[:,idx],'g')
    ax.set_xlim(0, 50)
    ax.set_ylim(0, 10)
    ax.set_xlabel('dt [ps]')
    
axs[0].set_ylabel('MSD [$\AA^2$]')
axs[0].legend(['CP2K 10-60ps','PiNet 10-60ps', 'PiNet 0-50ns'])
# plt.xscale('log')
# plt.yscale('log')
# plt.axis('equal')

axs[0].set_title('H*')
axs[1].set_title('O*')
axs[2].set_title('N*')
plt.savefig('diff-diag.png')

In [ ]:
f, axs = plt.subplots(1,3, figsize=[8,3])

rdfNA, rdfNB, rdfOA, rdfOB = [0]*4
for geo in ['a32b32i0', 'a16b16i16', 'a0b0i32']:
    rmid, rdf = np.loadtxt(f'../analyses/cp2k/{geo}-rho1.1551-rdf-10-60ps/HN_rdf.dat').T
    #axs[0].plot(rmid, rdf, ':', lw=1)
    rdfNA += rdf
    rmid, rdf = np.loadtxt(f'../analyses/cp2k/{geo}-rho1.1551-rdf-10-60ps/HO_rdf.dat').T
    #axs[0].plot(rmid, rdf, 'k:', lw=1)
    
    rdfOA += rdf
    rmid, rdf = np.loadtxt(f'../analyses/cp2k/{geo}-rho1.0753-rdf-10-60ps/HN_rdf.dat').T
    #axs[0].plot(rmid, rdf, 'k:', lw=1)

    rdfNB += rdf
    rmid, rdf = np.loadtxt(f'../analyses/cp2k/{geo}-rho1.1551-rdf-10-60ps/HO_rdf.dat').T
    axs[0].plot(rmid, rdf, 'k:', lw=1)

    rdfOB += rdf
axs[0].plot(rmid, rdfNA/3, 'r--')
axs[0].plot(rmid, rdfNB/3, 'r-')
axs[0].plot(rmid, rdfOA/3, 'k--')
axs[0].plot(rmid, rdfOB/3, 'k-')
axs[0].set_xlim(0.8,2)
axs[0].set_ylim(0,60)

rdfNA, rdfNB, rdfOA, rdfOB = [0]*4
for geo in ['a32b32i0', 'a16b16i16', 'a0b0i32']:
    rmid, rdf = np.loadtxt(f'../analyses/trial/{geo}-r1.16-rdf-10-60ps/HN_rdf.dat').T
    rdfNA += rdf
    rmid, rdf = np.loadtxt(f'../analyses/trial/{geo}-r1.16-rdf-10-60ps/HO_rdf.dat').T
    rdfOA += rdf
    rmid, rdf = np.loadtxt(f'../analyses/trial/{geo}-r1.08-rdf-10-60ps/HN_rdf.dat').T
    rdfNB += rdf
    rmid, rdf = np.loadtxt(f'../analyses/trial/{geo}-r1.08-rdf-10-60ps/HO_rdf.dat').T
    rdfOB += rdf
axs[1].plot(rmid, rdfNA/3, 'r--')
axs[1].plot(rmid, rdfNB/3, 'r-')
axs[1].plot(rmid, rdfOA/3, 'k--')
axs[1].plot(rmid, rdfOB/3, 'k-')
axs[1].set_xlim(0.8,2)
axs[1].set_ylim(0,60)

rdfNA, rdfNB, rdfOA, rdfOB = [0]*4
for geo in ['a32b32i0', 'a16b16i16', 'a0b0i32']:
    rmid, rdf = np.loadtxt(f'../analyses/trial/{geo}-r1.16-rdf-0-5ns/HN_rdf.dat').T
    rdfNA += rdf
    rmid, rdf = np.loadtxt(f'../analyses/trial/{geo}-r1.16-rdf-0-5ns/HO_rdf.dat').T
    rdfOA += rdf
    rmid, rdf = np.loadtxt(f'../analyses/trial/{geo}-r1.08-rdf-0-5ns/HN_rdf.dat').T
    rdfNB += rdf
    rmid, rdf = np.loadtxt(f'../analyses/trial/{geo}-r1.08-rdf-0-5ns/HO_rdf.dat').T
    rdfOB += rdf
axs[2].plot(rmid, rdfNA/3, 'r--')
axs[2].plot(rmid, rdfNB/3, 'r-')
axs[2].plot(rmid, rdfOA/3, 'k--')
axs[2].plot(rmid, rdfOB/3, 'k-')
axs[2].set_xlim(0.8,2)
axs[2].set_ylim(0,80)
axs[0].set_ylim(0,80)
axs[1].set_ylim(0,80)

axs[0].set_title('CP2K 10-60ps')
axs[1].set_title('PiNet 10-60ps')
axs[2].set_title('PiNet 0-50ns')
plt.savefig('rdf-diag.png')